# 0,1の長さ10の列を受け取って和を計算するLSTMを学習させる

In [1]:
%run rnn.py
sess.close()
tf.reset_default_graph()

train#0, train loss: 2.927592e+01
accuracy 0.000000
train#200, train loss: 4.677699e+00
train#400, train loss: 2.446417e+00
train#600, train loss: 2.618915e+00
train#800, train loss: 2.641371e+00
train#1000, train loss: 1.364590e+00
accuracy 0.034000
train#1200, train loss: 3.645671e-01
train#1400, train loss: 1.289618e-01
train#1600, train loss: 1.213474e-01
train#1800, train loss: 1.188318e-01
train#2000, train loss: 3.691995e-02
accuracy 0.320000
train#2200, train loss: 3.489457e-02
train#2400, train loss: 2.912243e-02
train#2600, train loss: 4.415308e-02
train#2800, train loss: 2.232399e-02
train#3000, train loss: 1.827935e-02
accuracy 0.546000
train#3200, train loss: 2.286179e-02
train#3400, train loss: 1.247409e-02
train#3600, train loss: 1.332075e-02
train#3800, train loss: 1.452823e-02
train#4000, train loss: 1.350400e-02
accuracy 0.398000
train#4200, train loss: 1.668620e-02
train#4400, train loss: 9.650654e-03
train#4600, train loss: 1.045391e-02
train#4800, train loss: 8.640

# tensorboardで学習経過を見てみる

In [10]:
%%script bash --bg --out sys.stdout --err sys.stderr
# 起動
tensorboard --logdir /tmp/tensorflow_log

Starting job # 0 in a separate thread.


### [tensorbordを開くにはココをクリック](http://0.0.0.0:6006)

In [11]:
%%script bash
# 終了
pid=$(ps augwx | grep tensorboard | grep -v grep | awk '{ print $2 }')
if [ -n "$pid" ]; then kill $pid; fi

# 学習後のLSTMに入力を与えてみる

In [12]:
# 上で構築したモデルにランダムな入力を与えて精度を計算する
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
with tf.Graph().as_default():
    input_ph      = tf.placeholder(tf.float32, [None, length_of_sequences, num_of_input_nodes], name="input")
    supervisor_ph = tf.placeholder(tf.float32, [None, num_of_output_nodes], name="supervisor")
    istate_ph     = tf.placeholder(tf.float32, [None, num_of_hidden_nodes * 2], name="istate")

    output_op, states_op, datas_op = inference(input_ph, istate_ph)
#     loss_op = loss(output_op, supervisor_ph)
#     training_op = training(loss_op)

    summary_op = tf.merge_all_summaries()
    init = tf.initialize_all_variables()

    with tf.Session() as sess:
        saver = tf.train.Saver()
        saver.restore(sess, "model.ckpt")
        
        #　テストデータの作成
        num_of_prediction_epochs = 10
        sequence_len = 10
        xs, ts = create_data(num_of_prediction_epochs, sequence_len)
        inputs, ts = np.array([[[y] for y in x] for x in xs]), np.array([[x] for x in ts])
        
        pred_dict = {
                input_ph:  inputs,
                supervisor_ph: ts,
                istate_ph:    np.zeros((num_of_prediction_epochs, num_of_hidden_nodes * 2)),
        }
        output= sess.run([output_op], feed_dict=pred_dict)

        def print_result (i, p, q):
            print([list(x)[0] for x in i])
            print("output: %f, correct: %d,  accurate: %s" % (p , q, (abs(p-q) < 0.05)[0]))
        [print_result(i, p, q)  for i, p, q in zip(inputs, output[0], ts)]

        opt = abs(output - ts)[0]
        total = sum([1 if x[0] < 0.05 else 0 for x in opt])
        print("accuracy %f" % (total/float(len(ts))))


[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0]
output: 5.993272, correct: 6,  accurate: True
[0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0]
output: 3.988194, correct: 4,  accurate: True
[0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0]
output: 4.991893, correct: 5,  accurate: True
[1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
output: 7.989611, correct: 8,  accurate: True
[0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0]
output: 4.992252, correct: 5,  accurate: True
[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0]
output: 3.988636, correct: 4,  accurate: True
[0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0]
output: 4.992630, correct: 5,  accurate: True
[0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0]
output: 4.991046, correct: 5,  accurate: True
[1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0]
output: 5.993834, correct: 6,  accurate: True
[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0]
output: 4.990329, correct: 5,  accurate: True
accuracy 1.000000
